In [66]:
import datetime
import pandas as pd
import numpy as np
import random
from scipy import sparse
from scipy.special import digamma
from scipy.sparse import csr_matrix
from time import time
from functools import reduce
# from guppy import hpy

K = 5
numIter = 100
top_k = 100

path = r"D:/Download/data/"
# datasetName = "lastfm_sample_190617/"
datasetName = r"\go_ny"
trainFile = path + datasetName + r"\train.csv"
validationFile = path + datasetName + r"\validation.csv"
testFile = path + datasetName + r"\test.csv"
train = pd.read_csv(trainFile, header=None)
validation = pd.read_csv(validationFile, header=None)
test = pd.read_csv(testFile, header=None)

train.columns = ["user_id", "loc_id", "count"]
validation.columns = ["user_id", "loc_id", "count"]
test.columns = ["user_id", "loc_id", "count"]

def function01(data, train):
	array = []
	for index, row in data.iterrows():
		temp = [[row['user_id'], row['loc_id'], train]] * row['count']
		array.extend(temp)

	df = pd.DataFrame(array)
	# df = df.sample(frac=0.4, replace=True, random_state=1)
	df.columns = ["user_id", "loc_id", "train"]
	return df

train_df = function01(train,1)
validation_df = function01(validation,1)
test_df = function01(test, 0)
# train_user_set = set(train_df['user_id'].tolist())
# validation_user_set = set(validation_df['user_id'].tolist())
# user_of_interest = train_user_set.intersection(validation_user_set)


# train_df = train_df[train_df['user_id'].isin(user_of_interest)]
# validation_df = validation_df[validation_df['user_id'].isin(user_of_interest)]
# test_df = test_df[test_df['user_id'].isin(user_of_interest)]

train_df = pd.concat([train_df,validation_df])
test_df = test_df[test_df['loc_id'].isin(train_df['loc_id'].tolist())]

df = pd.concat([train_df, test_df])

loc_df = pd.DataFrame(df['loc_id'].unique())
loc_df['index1'] = loc_df.index
loc_df.columns = ['loc_id', 'index1']

df = pd.merge(df, loc_df, how='inner', left_on=['loc_id'], right_on=['loc_id']) \
	[['user_id', 'index1', 'train']]
df.columns = ['user_id', 'loc_id', 'train']
df = df.sample(frac=1, random_state=123).sort_values(by=['user_id', 'train'], ascending=[1, 0])

# print(df[df['user_id'] == 100])
num_record = len(df)
print(num_record)


df_inference = df[['user_id', 'loc_id', 'train']]
temp = np.random.dirichlet(np.ones(K) * 1000, len(df)).tolist()
temparray = [np.array(row) for row in temp]
df_inference['phi'] = temparray

nodelist = list(set(df_inference.user_id.tolist()))
venuelist = list(set(df_inference.loc_id.tolist()))

venueSize = len(venuelist)

print(venueSize)
print(len(nodelist))

roleVenue = np.random.dirichlet(np.ones(venueSize), K)
#roleVenue = np.ones([K, venueSize]) / venueSize

tempp = (np.random.dirichlet(np.ones(K) * 1000, len(nodelist)) * K).tolist()
tempparray = [np.array(row) for row in tempp]
seriesgamma = tempparray
dfgamma = pd.DataFrame(list(zip(nodelist, seriesgamma)), columns=['user_id', 'gamma'])

df_table = pd.merge(df_inference, dfgamma, how='left', left_on=['user_id'], right_on=['user_id']) \
	[['user_id', 'loc_id', 'phi', 'train', 'gamma']]

print("Calculate the history")
history = [[0]] * num_record
start = time()
temp_his = np.zeros(venueSize)
temp_count = 0
repeated = np.zeros(num_record)
prev_user = -1
history_index = 0

for r in zip(df_table['user_id'], df_table['train'], df_table['loc_id']):
	# if history_index % 1000 == 0:
	# 	print(history_index)
	if prev_user == -1 or r[0] != prev_user:
		temp_his = csr_matrix((1, venueSize), dtype=int)
		temp_count = 0
		history[history_index] = csr_matrix(temp_his)
	else:
		if temp_count == 0:  # added for divide by zero
			history[history_index] = temp_his
		else:
			history[history_index] = temp_his / temp_count
	if r[1] == 1:
		temp_his[(0, r[2])] += 1
		temp_count += 1
	else:
		if temp_his[(0, r[2])] > 0:
			repeated[history_index] = 1
	prev_user = r[0]
	history_index += 1
df_table['history'] = history
df_table['repeated'] = repeated

# print(df_table[df_table['user_id'] == 0])

print(time() - start)
print("done")
# print(df_table['history'])

df_table_training = df_table[df_table.train == 1]

# print(len(set(df_table[df_table.train == 1]['loc_id'].tolist())))
df_table_testing = df_table[df_table.train == 0]
df_table_backup = df_table
df_table = df_table_training


def toSparseMatrix(x):
	I = range(K)
	J = np.full(K, x.loc_id)
	V = x.phi
	return sparse.coo_matrix((V, (I, J)), shape=(K, venueSize))


def updateRoleVenue(x):
	roleVenue = np.ones([K, len(venuelist)]) / len(venuelist) /(K-1)

	a = x.groupby('loc_id')['phi'].apply(lambda x: x.sum()).sort_index()
	b = np.array(a.tolist())
	b = b.T
#	roleVenue[:,:-(roleVenue.shape[1] - b.shape[1])] += b 
	roleVenue += b
#	roleVenue = b
	roleVenue /= roleVenue.sum(axis=1)[:, np.newaxis]

#	roleVenue = 0.8* np.array([g, ] * K) + 0.2 * roleVenue
	roleVenue[K-1] = g    
	return roleVenue


df_table['temp'] = df_table['loc_id'].apply(lambda x: roleVenue[:, x])


def f_test(x):
	temp_history = x[2]
	temp_loc = x[1]
	temp = x[0]
	temp[0] = temp_history[(0, temp_loc)]
	return temp


input_temp = df_table[['temp', 'loc_id', 'history']].values
df_table['temp'] = np.apply_along_axis(f_test, 1, input_temp).tolist()
df_table['temp'] = df_table['temp'].apply(lambda x: np.array(x))


def multiplyArray_backup(x):
	return tuple(np.multiply(x['temp'], x['gamma']))


def multiplyArray(x):  ## x[1] gamma
	temp = digamma(x[1])
	temp = np.exp(temp)
	a = np.multiply(x[0], temp)
	return a / (a.sum())


input_temp = df_table[['temp', 'gamma']].values
df_table['phi'] = np.apply_along_axis(multiplyArray, 1, input_temp).tolist()
df_table['phi'] = df_table['phi'].apply(lambda x: np.array(x))

# roleVenue = updateRoleVenue(df_table)

vf_test = np.vectorize(f_test)

### update global
g = df_table.groupby(['loc_id'])['user_id'].apply(np.unique).apply(lambda x: len(x)).sort_index()
# g = df_table.groupby('loc_id')['user_id'].apply(lambda x: x.count()).sort_index()
g = np.array(g.tolist())
g = g.T
g = g / float(sum(g))

print("optimize")

for i in range(numIter):
	print("iter " + str(i + 1))

	dfgamma = df_table.groupby(['user_id'])['phi'].apply(lambda x: x.sum() + np.ones(K) / K).to_frame().reset_index()
	dfgamma.columns = ['user_id', 'gamma']

	df_table['temp'] = df_table['loc_id'].apply(lambda x: roleVenue[:, x])

	input_temp = df_table[['temp', 'loc_id', 'history']].values
	df_table['temp'] = np.apply_along_axis(f_test, 1, input_temp).tolist()
	df_table['temp'] = df_table['temp'].apply(lambda x: np.array(x))

	input_temp = df_table[['temp', 'gamma']].values
	df_table['phi'] = np.apply_along_axis(multiplyArray, 1, input_temp).tolist()
	df_table['phi'] = df_table['phi'].apply(lambda x: np.array(x))

	df_inference = df_table[['user_id', 'loc_id', 'phi', 'train', 'history']]

	df_table = pd.merge(df_inference, dfgamma, how='left', left_on=['user_id'], right_on=['user_id']) \
		[['user_id', 'loc_id', 'phi', 'gamma', 'train', 'history']]

	roleVenue = updateRoleVenue(df_table)


print("Optimize done")
valid_user = df_table_backup.groupby('user_id')['train'].mean()
valid_user = valid_user[(valid_user > 0.001) & (valid_user < 0.999)]
valid_user = list(set(valid_user.index))

test = pd.DataFrame(columns=['user_id', 'loc_id', 'repeated', 'rank'])


df_inference_testing = df_table_testing[['user_id', 'loc_id', 'phi', 'train', 'history', 'repeated']]
df_table_testing = pd.merge(df_inference_testing, dfgamma, how='left', left_on=['user_id'], right_on=['user_id']) \
	[['user_id', 'loc_id', 'phi', 'gamma', 'train', 'history', 'repeated']]

df_table_testing = df_table_testing[df_table_testing['user_id'].isin(valid_user)].reset_index()
in_top = np.zeros(len(df_table_testing))
ranking_avg = np.zeros(len(df_table_testing))
print("calculate ranking")

def rates_to_exp_order(rates, argsort, exp_order, M):
    prev_score = 0
    prev_idx = 0
    prev_val = rates[argsort[0]]
    for i in range(1, M):
        if prev_val == rates[argsort[i]]:
            continue

        tmp = 0
        for j in range(prev_idx, i):
            exp_order[argsort[j]] = prev_score + 1
            tmp += 1

        prev_score += tmp
        prev_val = rates[argsort[i]]
        prev_idx = i

    # For the last equalities
    for j in range(prev_idx, i + 1):
        exp_order[argsort[j]] = prev_score + 1

avg_gamma = dfgamma.mean().gamma

for index, row in df_table_testing.iterrows():
#	if np.isnan(np.min(row['gamma']))  :
#		print("here")
#	else:
	roleVenue2 = np.array(roleVenue)
	if np.isnan(np.min(row['gamma'])) :
		roleVenue2[0] = np.ones(venueSize)/venueSize
		array = np.dot(avg_gamma/ sum(avg_gamma), roleVenue2 )
	else:
		roleVenue2[0] = row['history'].toarray()
		array = np.dot(row['gamma'] / sum(row['gamma']), roleVenue2)
	scores =  array
	argsort = np.argsort(-scores)
	# exp_order = np.zeros(scores.shape)
	# rates_to_exp_order(scores, argsort, exp_order, len(scores))
	# ranks = exp_order
	order = array.argsort()
	ranks = venueSize - order.argsort()
	if ranks[row['loc_id']] <= top_k:
		# user_true_prediction_count[nodelist.index(row['user_id'])] += 1
		in_top[index] = 1
	ranking_avg[index] = ranks[row['loc_id']]
	# is_repeated = 0
	# if row['history'][(0, row['loc_id'])] > 0:
	# 	is_repeated = 1
	# test.loc[len(ranking_avg) - 1] = [row['user_id'], row['loc_id'], is_repeated, ranks[row['loc_id']]]
	# user_prediction_count[nodelist.index(row['user_id'])] += 1


print(reduce(lambda x, y: x + y, ranking_avg) / len(ranking_avg))
print(K)
df_table_testing['in_top'] = in_top
df_table_testing['rank'] = ranking_avg

print(np.mean(df_table_testing.groupby('user_id')['in_top'].mean().values))
print(sum(df_table_testing['in_top']) / len(df_table_testing))

print(np.mean(df_table_testing[df_table_testing['repeated'] == 1].groupby('user_id')['in_top'].mean().values))
print(np.mean(df_table_testing[df_table_testing['repeated'] == 0].groupby('user_id')['in_top'].mean().values))

#df_table_testing[['user_id', 'loc_id', 'rank']].to_csv(
#	"D:/Research/Project/LDA/results/"+datasetName[:-1]+"_K"+str(K)+"_I"+str(numIter)+".csv")

75007
7442
1828
Calculate the history
20.323660612106323
done
optimize
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
Optimize done
calculate ranking
1499.9848880809
5
0.4348153818436582
0.46358368367230995
0.9682250329389667
0.18629503243099935


C:\Users\ducnm\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
C:\Users\ducnm\Anaconda3\lib\site-packages\ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ducnm\Anaconda3\lib\site-packages\ipykernel_launcher.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ducnm\Anaconda3\lib\site-packages\ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [13]:
df_table_testing.shape

(12733, 10)

In [14]:
roleVenue.shape

(5, 7578)

In [15]:
loc_df.shape

(7578, 2)

In [17]:
train.tail()

,user_id,loc_id,count
63090,2149,1078,1
63091,2149,4546,1
63092,2149,6411,1
63093,2150,233,1
63094,2592,7705,0


In [18]:
train.loc_id.min()

0

In [19]:
test.loc_id.min()

nan

In [27]:
a = test.loc_id.to_list()
b = train.loc_id.to_list()
c = validation.loc_id.to_list()

In [35]:
import datetime
import pandas as pd
import numpy as np
import random
from scipy import sparse
from scipy.special import digamma
from scipy.sparse import csr_matrix
from time import time
from functools import reduce
# from guppy import hpy

K = 5
path = r'C:\Users\Preload\Downloads\repeat_consumption_data\data'
# datasetName = "lastfm_sample_190617/"
datasetName = r"\go_sf"
trainFile = path + datasetName + r"\train.csv"
validationFile = path + datasetName + r"\validation.csv"
testFile = path + datasetName + r"\test.csv"
train = pd.read_csv(trainFile, header=None)
validation = pd.read_csv(validationFile, header=None)
test = pd.read_csv(testFile, header=None)

train.columns = ["user_id", "loc_id", "count"]
validation.columns = ["user_id", "loc_id", "count"]
test.columns = ["user_id", "loc_id", "count"]

def function01(data, train):
	array = []
	for index, row in data.iterrows():
		temp = [[row['user_id'], row['loc_id'], train]] * row['count']
		array.extend(temp)

	df = pd.DataFrame(array)
	# df = df.sample(frac=0.4, replace=True, random_state=1)
	df.columns = ["user_id", "loc_id", "train"]
	return df


train_df = pd.concat([function01(train, 1), function01(validation, 1)])
test_df = function01(test, 0)
#test_df = test_df[test_df['loc_id'].isin(train_df['loc_id'].tolist())]

df = pd.concat([train_df, test_df])

loc_df = pd.DataFrame(df['loc_id'].unique())
loc_df['index1'] = loc_df.index
loc_df.columns = ['loc_id', 'index1']

df = pd.merge(df, loc_df, how='inner', left_on=['loc_id'], right_on=['loc_id']) \
	[['user_id', 'index1', 'train']]
df.columns = ['user_id', 'loc_id', 'train']
df = df.sample(frac=1, random_state=123).sort_values(by=['user_id', 'train'], ascending=[1, 0])

# print(df[df['user_id'] == 100])
num_record = len(df)
print(num_record)


df_inference = df[['user_id', 'loc_id', 'train']]
temp = np.random.dirichlet(np.ones(K) * 1000, len(df)).tolist()
temparray = [np.array(row) for row in temp]
df_inference['phi'] = temparray

nodelist = list(set(df_inference.user_id.tolist()))
venuelist = list(set(df_inference.loc_id.tolist()))

venueSize = len(venuelist)

print(venueSize)
print(len(nodelist))

roleVenue = np.random.dirichlet(np.ones(venueSize), K)
# roleVenue = np.ones([K, venueSize]) / venueSize

tempp = (np.random.dirichlet(np.ones(K) * 1000, len(nodelist)) * K).tolist()
tempparray = [np.array(row) for row in tempp]
seriesgamma = tempparray
dfgamma = pd.DataFrame(list(zip(nodelist, seriesgamma)), columns=['user_id', 'gamma'])

df_table = pd.merge(df_inference, dfgamma, how='left', left_on=['user_id'], right_on=['user_id']) \
	[['user_id', 'loc_id', 'phi', 'train', 'gamma']]

print("Calculate the history")
history = [[0]] * num_record
start = time()
temp_his = np.zeros(venueSize)
temp_count = 0
repeated = np.zeros(num_record)
prev_user = -1
history_index = 0

for r in zip(df_table['user_id'], df_table['train'], df_table['loc_id']):
	if history_index % 1000 == 0:
		print(history_index)
	if prev_user == -1 or r[0] != prev_user:
		temp_his = csr_matrix((1, venueSize), dtype=int)
		temp_count = 0
		history[history_index] = csr_matrix(temp_his)
	else:
		if temp_count == 0:  # added for divide by zero
			history[history_index] = temp_his
		else:
			history[history_index] = temp_his / temp_count
	if r[1] == 1:
		temp_his[(0, r[2])] += 1
		temp_count += 1
	else:
		if temp_his[(0, r[2])] > 0:
			repeated[history_index] = 1
	prev_user = r[0]
	history_index += 1
df_table['history'] = history
df_table['repeated'] = repeated

# print(df_table[df_table['user_id'] == 0])

print(time() - start)
print("done")

138185
7702
2593
Calculate the history
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
44.53668236732483
done


In [44]:
df_table.groupby('loc_id')['phi'].apply(lambda x: x.sum())

loc_id
0       [23.28637061614094, 18.366709121101437, 12.382...
1       [44.46251440322267, 1.34675459115368, 56.68962...
2       [3.860743654636549, 6.141398977201452, 9.47442...
3       [5.520649369264185, 11.050822846488794, 7.7783...
4       [483.11373043817696, 15.392046078178481, 62.71...
5       [0.6983124366359293, 8.029501581926413, 3.0914...
6       [0.0, 0.3388095901102809, 2.532627370102498, 1...
7       [21.122535724393686, 1.6572948655514679, 2.189...
8       [261.79565693703927, 48.11563505616352, 2.7581...
9       [39.71398642491456, 6.896212510010361, 72.4936...
10      [15.58432446078675, 3.1416334961922887, 6.8215...
11      [28.254244667799515, 10.456585780034722, 20.41...
12      [5.6499901096144525, 14.499855766402955, 1.230...
13      [48.873115205508206, 9.046754831939369, 60.165...
14      [17.547655888541097, 3.415911117473491, 22.884...
15      [3.8085321425788656, 2.6962902981322685, 7.794...
16      [18.17399494506052, 0.34267971709058853, 2.357...
17     

In [46]:
tttemp = df_table.groupby('loc_id')['phi'].apply(lambda x: x.sum())
type(tttemp)

pandas.core.series.Series

In [47]:
tttemp

loc_id
0       [23.28637061614094, 18.366709121101437, 12.382...
1       [44.46251440322267, 1.34675459115368, 56.68962...
2       [3.860743654636549, 6.141398977201452, 9.47442...
3       [5.520649369264185, 11.050822846488794, 7.7783...
4       [483.11373043817696, 15.392046078178481, 62.71...
5       [0.6983124366359293, 8.029501581926413, 3.0914...
6       [0.0, 0.3388095901102809, 2.532627370102498, 1...
7       [21.122535724393686, 1.6572948655514679, 2.189...
8       [261.79565693703927, 48.11563505616352, 2.7581...
9       [39.71398642491456, 6.896212510010361, 72.4936...
10      [15.58432446078675, 3.1416334961922887, 6.8215...
11      [28.254244667799515, 10.456585780034722, 20.41...
12      [5.6499901096144525, 14.499855766402955, 1.230...
13      [48.873115205508206, 9.046754831939369, 60.165...
14      [17.547655888541097, 3.415911117473491, 22.884...
15      [3.8085321425788656, 2.6962902981322685, 7.794...
16      [18.17399494506052, 0.34267971709058853, 2.357...
17     

In [48]:
a_test = np.zeros(K,venueSize)

TypeError: data type not understood

In [51]:
a_test = np.zeros([K, len(venuelist)])

In [55]:
np.array(tttemp.tolist())

array([[23.28637062, 18.36670912, 12.38277047,  1.43972206,  9.52442773],
       [44.4625144 ,  1.34675459, 56.6896281 ,  0.97461546, 15.52648744],
       [ 3.86074365,  6.14139898,  9.47442811, 11.43245757,  1.09097168],
       ...,
       [ 0.        ,  0.23201427,  0.29907363,  0.31349133,  0.15542077],
       [ 0.99822904,  0.23879765,  0.14391533,  0.26630539,  0.35275259],
       [ 0.        ,  0.14132872,  0.42536259,  0.07452712,  0.35878157]])

In [57]:
b = np.array([[1,2,3,],[4,5,6]])

In [59]:
b[:,:-2] 

array([[1],
       [4]])

In [65]:
dfgamma

,user_id,gamma
0,0,"[50.723078325747835, 0.20212909998238857, 0.20..."
1,1,"[154.19526869954848, 71.96548909837259, 0.2041..."
2,2,"[33.33350636262454, 55.40070537525056, 15.2854..."
3,3,"[48.10759163259252, 12.793559541818237, 154.32..."
4,4,"[0.2, 6.168128972849693, 0.2030536043760028, 0..."
5,5,"[59.63413682853028, 4.446832128937746, 99.9424..."
6,6,"[109.71604322268881, 46.496449561467955, 0.202..."
7,7,"[3.117791581804049, 0.2018042861453979, 0.2031..."
8,8,"[88.14248493998208, 50.58098270042765, 78.7096..."
9,9,"[10.59236734554414, 28.151135209956756, 0.2052..."


In [17]:
df_table_testing

,index,user_id,loc_id,phi,gamma,train,history,repeated,in_top,rank
0,0,1,64,"[0.2545475797182508, 0.24369952008677417, 0.24...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,3.0
1,1,1,1677,"[0.2400991854452686, 0.24640455413848528, 0.25...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",0.0,0.0,350.0
2,2,1,102,"[0.23755194568599855, 0.2516755599639376, 0.25...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,18.0
3,3,1,64,"[0.2686727236902478, 0.2692413935566609, 0.226...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,3.0
4,4,1,63,"[0.24895030370496699, 0.2535338452700426, 0.24...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,1.0
5,5,1,395,"[0.24797363919383508, 0.25379430868762193, 0.2...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",0.0,0.0,187.0
6,6,1,243,"[0.25993174464070956, 0.24887418157489216, 0.2...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",0.0,0.0,220.0
7,7,1,64,"[0.2466204312161858, 0.2541245408562041, 0.256...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,3.0
8,8,1,64,"[0.2518590761409029, 0.25255678602429693, 0.24...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",1.0,1.0,3.0
9,9,1,6306,"[0.2515335633730384, 0.24857263297977694, 0.25...","[155.0817419828932, 21.41980849326475, 33.9139...",0,"(0, 2)\t0.004132231404958678\n (0, 4)\t0.00...",0.0,0.0,3265.0


In [103]:
venueSize

7702

In [5]:
dfgamma.gamma = dfgamma.gamma/dfgamma.gamma.sum()

In [6]:
dfgamma

,user_id,gamma
0,0,"[0.0011895650490396435, 0.0028065535496086176,..."
1,1,"[0.003649093969653999, 0.00039437903533793813,..."
2,2,"[0.0008027367804504259, 0.0013326529738500867,..."
3,3,"[0.001270497490394149, 0.012760168027925524, 0..."
4,4,"[4.7237880951326285e-06, 9.818261830924307e-06..."
5,5,"[0.0014377625703421804, 0.0017005027996969358,..."
6,6,"[0.0026172459384284767, 0.0028909810299152195,..."
7,7,"[7.485961203685588e-05, 1.0149298418156885e-05..."
8,8,"[0.0020611021885253763, 0.0022948576114172276,..."
9,9,"[0.0002380384815268038, 2.048764160425648e-05,..."


In [3]:
df_table_testing

,index,user_id,loc_id,phi,gamma,train,history,repeated,in_top,rank
0,0,1,6,"[0.19896113301653037, 0.186239984035793, 0.209...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,1.0
1,1,1,7,"[0.19855553206628762, 0.1938570780793909, 0.21...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,3.0
2,2,1,12,"[0.20455019582503942, 0.20987818376306458, 0.2...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,4.0
3,3,1,7,"[0.20164018717491633, 0.21104389565964154, 0.1...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,3.0
4,4,1,1019,"[0.19816623792105945, 0.19963327931145364, 0.1...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,51.0
5,5,1,6,"[0.19425340397674246, 0.1983453637383135, 0.20...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,1.0
6,6,1,21,"[0.2105527970224037, 0.19021426278116665, 0.19...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,31.0
7,7,1,6,"[0.1959706725637655, 0.19921865650174944, 0.19...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,1.0
8,8,1,5567,"[0.19566902523557603, 0.20624033253190976, 0.2...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",0.0,0.0,2748.0
9,9,1,6,"[0.19242333831167946, 0.19887783773355588, 0.2...","[155.68551955795135, 10.326556093998201, 76.57...",0,"(0, 0)\t0.004132231404958678\n (0, 1)\t0.00...",1.0,1.0,1.0


In [3]:
df_table.groupby('loc_id')['phi'].apply(lambda x: x.sum()).sort_index()

loc_id
0       [23.452876539687804, 12.808779973688713, 11.46...
1       [42.142501519580335, 20.593361403448483, 31.07...
2       [3.762095023763477, 16.074856659339673, 0.0830...
3       [5.61088096407408, 8.203701898197172, 3.266677...
4       [337.82387531056554, 95.50284459906119, 364.48...
5       [0.7555619993181917, 22.94283001452389, 0.0001...
6       [0.0, 5.840258804915607, 1.402246756731315, 1....
7       [19.51995608799732, 24.979303875347842, 20.586...
8       [256.16802213917225, 40.72989734467165, 62.481...
9       [34.55808689966276, 54.22421211661695, 50.4472...
10      [15.48280788219047, 9.31836678119532, 17.65849...
11      [26.792432586069097, 27.166518413648582, 8.909...
12      [5.858115416263597, 12.83969898926369, 7.84811...
13      [41.62835086075422, 42.85778811879252, 11.7559...
14      [17.277367672452876, 29.923223972155906, 2.216...
15      [3.0233654603065125, 17.019248179031802, 4.566...
16      [18.917974425048858, 3.404452606292554, 2.0435...
17     

In [29]:
pop_loc = np.ones(venueSize) / len(venuelist)
for index, row in df_table.iterrows():
    if row['train'] > 0:
        pop_loc[row['loc_id']] += 1
pop_loc = pop_loc/sum(pop_loc)

In [32]:
pop_loc.shape

(7702,)

In [33]:
pop_loc

array([5.35588816e-04, 9.80538635e-04, 2.63675037e-04, ...,
       1.06982751e-09, 1.06982751e-09, 1.06982751e-09])

In [3]:
df_table.groupby('loc_id')['phi'].apply(lambda x: x.sum()).sort_index()

loc_id
0       [23.230961544289265, 30.36306978465666, 8.2905...
1       [41.11794213771568, 39.53193281411946, 27.4985...
2       [3.659690870804515, 20.566779888100054, 5.7657...
3       [5.433200533219871, 20.44549662172281, 2.60398...
4       [322.03002262995875, 68.08682043791426, 292.90...
5       [0.8399808760303238, 15.307035044612528, 0.908...
6       [0.0, 2.528201369059399, 1.7401510119260322, 4...
7       [19.08181436883521, 36.590096355927116, 0.7338...
8       [255.0008620326137, 15.07241084517093, 62.2748...
9       [32.04686983064131, 85.27126417115659, 3.18276...
10      [14.922101462491103, 27.196322685066917, 1.222...
11      [25.17339103593253, 56.75451058791356, 0.00011...
12      [5.777567597999767, 28.130821393475426, 2.6141...
13      [40.88780229163812, 65.17522782616541, 7.26906...
14      [17.309965699762238, 25.366983434646382, 1.226...
15      [3.0766163812399614, 12.964407158230204, 6.608...
16      [18.824470277095667, 5.175387967113224, 0.0001...
17     

In [5]:
df_table_backup.groupby('user_id')['train'].mean()

user_id
0       1.000000
1       0.793443
2       0.945392
3       1.000000
4       0.666667
5       0.857143
6       0.913669
7       1.000000
8       0.748848
9       1.000000
10      1.000000
11      1.000000
12      1.000000
13      1.000000
14      1.000000
15      0.971193
16      1.000000
17      0.991736
18      0.969697
19      0.673684
20      1.000000
21      0.464286
22      0.984496
23      1.000000
24      1.000000
25      0.777778
26      1.000000
27      0.691176
28      1.000000
29      0.928571
          ...   
2563    0.000000
2564    0.000000
2565    0.000000
2566    0.000000
2567    0.000000
2568    0.000000
2569    0.000000
2570    0.000000
2571    0.000000
2572    0.000000
2573    0.000000
2574    0.000000
2575    0.000000
2576    0.000000
2577    0.000000
2578    0.000000
2579    0.000000
2580    0.000000
2581    0.000000
2582    0.000000
2583    0.000000
2584    0.000000
2585    0.000000
2586    0.000000
2587    0.000000
2588    0.000000
2589    0.000000
2590  

In [2]:
roleVenue[K-1] = g

In [44]:
g = df_table.groupby(['loc_id'])['train'].apply(lambda x: x.count()).sort_index()
# g = df_table.groupby(['loc_id'])['user_id'].apply(np.unique).apply(lambda x: len(x)).sort_index()
g = np.array(g.tolist())
g = g.T
g = g / float(sum(g))
g[:10]

array([5.35592159e-04, 9.80545645e-04, 2.63676140e-04, 2.55436260e-04,
       7.20165457e-03, 2.05996984e-04, 7.41589143e-05, 7.00389746e-04,
       3.23827259e-03, 1.43373901e-03])